In [1]:
#!/usr/bin/python2.7
# -*- coding: utf-8 -*-
from Kuytu.wikiLog import KuytuLog
from Kuytu.WikiDumpParser import WikiDumpParser
import Kuytu.file_commander as K_fc
import Kuytu.Analyzer as Analyzer


import json
from collections import Counter

In [2]:
log = KuytuLog('BlackBoxCode','r')
output_path = log.get_output_path()
StandartPagesXMLPath = output_path + 'BulkData/withInfoBoxPages_bulkXML.xml'

Articles_with_BK = K_fc.read_XML(StandartPagesXMLPath)

In [3]:
from Kuytu.article_cleaner_kit import clean_InfoBoxBulk #* # clean_InfoBoxBulk, set_BK_fieldsMaps, get_BK_fieldsMaps

for i,a in enumerate(Articles_with_BK):
    Articles_with_BK[i].set_infoBox_clean( infoBox_clean = clean_InfoBoxBulk( a.get_infoBoxText()) )    

In [4]:
Articles_with_BK_clean =  filter(lambda a: a.get_cleanInfoBox() != None , Articles_with_BK)

In [5]:

uniq_types = map(lambda a : a.get_infoBox_type() , Articles_with_BK_clean)
c = Counter( uniq_types )
uniq_types_histogram = list(c.items())


K_fc.save_Graph( output_path = output_path
                ,data = uniq_types_histogram
                ,min_repetition = 100
                ,title = 'Uniq-BK-Types-Hit-Counts-Graph-Clean(>100)' )

In [6]:
print '#Articles parsed clean', len(Articles_with_BK_clean)

#Articles parsed clean 178376


In [7]:
# Our interested Info Box Types
Interested_Info_Box_Types = [ u'Hakem' ,u'Manken' ,u'Makam Sahibi' ,u'Filozof' ,u'Bilim Insanı',u'Güreşçi' 
                             ,u'Bilim Adamı' ,u'Sporcu' ,u'Buz Patencisi',u'Asker' 
                             ,u'Voleybolcu' ,u'Sanatçı',u'Futbolcu' ,u'Oyuncu' 
                             ,u'Müzik Sanatçısı' ,u'Yazar' ,u'Kraliyet' ,u'Tenis Sporcu' ,u'Profesyonel Güreşçi'
                             ,u'Kişi' ,u'Basketbolcu']
Interested_articles_with_BK_clean =  filter(lambda a: a.get_infoBox_type() in Interested_Info_Box_Types , Articles_with_BK_clean)


In [8]:
print '#Articles interested categories', len(Interested_articles_with_BK_clean)

#Articles interested categories 53628


In [9]:
seperated_interested_articles = Analyzer.seperate_articles_according_to_type(Interested_articles_with_BK_clean)

In [10]:
seperated_interested_articles[u"Bilim Insanı"] += seperated_interested_articles[u"Bilim Adamı"]
del seperated_interested_articles[u"Bilim Adamı"]

In [11]:
map_ = {
    u'Manken' :{
        "yer" : "doğumyeri"
    },
    u'Hakem' : {
        "etkinyıl" : "aktifyıl",
        "yıl" : "aktifyıl",
        "yer" : "doğumyeri"
    }
}         
seperated_interested_articles = Analyzer.datafield_map(map_, seperated_interested_articles)

In [12]:
total_count_data, count_data = Analyzer.data_field_counter(seperated_interested_articles)      

In [13]:
import Kuytu.Analyzer as Analyzer
output_path = log.get_output_path()
total_article_count = float(len(Interested_articles_with_BK_clean))
subcats_article_counts =  { type_ :float(len(list_)) for type_,list_ in seperated_interested_articles.items()}

Analyzer.save_count_data( path = output_path + 'Counts'
                        , total_count_data =  total_count_data
                        , count_data = count_data
                        , total_first_n = 100
                        , subcat_first_n = 20
                        , total_article_count = total_article_count
                        , subcats_article_counts = subcats_article_counts )

In [14]:
interested_datafields =  {
    u"Hakem" :               ["ad","doğumtarihi" ,"turnuva"    ,"aktifyıl","doğumyeri"],
    u"Sporcu" :              ["ad","doğumtarihi" ,"spor"       ,"doğumyeri","ülke"],
    u"Kraliyet"  :           ["ad","doğumtarihi" ,"hanedan"    ,"hükümsüresi","ölümtarihi"],
    u"Voleybolcu" :          ["ad","doğumtarihi" ,"pozisyon"   ,"doğumyeri"],
    u"Manken" :              ["ad","doğumtarihi" ,"ulus"       ,"doğumyeri"],
    u"Oyuncu" :              ["ad","doğumtarihi" ,"meslek"     ,"yer","ölümtarihi"],
    u"Asker" :               ["ad","doğumtarihi" ,"rütbesi"    ,"doğumyeri","ölümtarihi"],
    u"Makam Sahibi" :        ["ad","doğumtarihi" ,"makam"      ,"doğumyeri"],
    u"Buz Patencisi" :       ["ad","doğumtarihi" ,"ülke"       ,"koç"],
    u"Profesyonel Güreşçi" : ["ad","doğumtarihi" ,"doğumyeri"],
    u"Kişi" :                ["ad","doğumtarihi" ,"meslek"     ,"doğumyeri","ölümtarihi"],
    u"Futbolcu" :            ["ad","doğumyeri"   ,"pozisyon"   ,"doğumtarihi"],
    u"Tenis Sporcu" :        ["ad","doğumyeri"   ,"oyunstili"  ,"doğumtarihi"],
    u"Bilim Insanı" :        ["ad","doğumyeri"   ,"dalı"       ,"doğumtarihi"],
    u"Filozof" :             ["ad","doğumyeri"   ,"doğumtarihi","çağ"],
    u"Basketbolcu" :         ["ad","doğumyeri"   ,"pozisyon"   ,"doğumtarihi"],
    u"Güreşçi" :             ["ad","doğumyeri"   ,"doğumtarihi","ölümtarihi"],
    u"Yazar" :               ["ad","doğumyeri"   ,"meslek"     ,"doğumtarihi","ölümtarihi"],
    u"Müzik Sanatçısı":      ["ad","artalan"     ,"tarz"       ,"etkinyıllar","meslek"],
    u"Sanatçı" :             ["ad","alanı"       ,"ölümtarihi" ,"ölümyeri"]
}
seperated_interested_articles2 = Analyzer.hold_interested_datafields(interested_datafields, seperated_interested_articles)

In [15]:
import xlsxwriter
def export_2_excel_for_check(file_name,articles_data,clomuns):
    '''
        to see the regex result and errors in data
    '''
    # Create a workbook and add a worksheet.
    workbook = xlsxwriter.Workbook(file_name)
    for type_ in articles_data.keys():
        worksheet = workbook.add_worksheet(type_)
        row = 0
        mapp_ = {}        
        for j,a in enumerate(articles_data[type_]):
            col = 0
            if j == 0:
                for i,key in enumerate(clomuns[type_]):
                    mapp_[key] = col+i
                    worksheet.write(row, col+i,  key.decode('utf-8')  )
                row += 1

                
            for i,(key,value) in enumerate(a.get_cleanInfoBox().items()):
                if value == None:
                    continue
                col = 0
                worksheet.write(row, mapp_[key], value.decode('utf-8'))
            
            row += 1
    workbook.close()

In [16]:
export_2_excel_for_check('../dememe.xlsx',seperated_interested_articles2,interested_datafields)

In [17]:
a = 'asdasda asdda'

In [18]:
a.title()
import re

In [19]:
orj = '{{Ölüm yılı ve yaşı|1947|1889}}'
reg = "[^\|]*ve yaşı\|(\d*)\|\d*"
match = re.search(r"[^\|]*ve yaşı\|(\d*)\|\d*", orj)
print match.group(1)  

1947
